<a href="https://colab.research.google.com/github/Dr-Carlos-Villasenor/Taller_CNN/blob/main/D2_Red_Neuronal_Densa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Red Neuronal Densa (Perceptrón Multicapa MLP)
## Dr. Carlos Villaseñor

Paso 1. Correr paqueterias

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Paso 2. Corre el siguiente bloque con diferentes funciones de activación

In [ ]:
# Funciones de activación
def linear(z, derivative=False):
    a = z
    if derivative:
        da = 1
        return a, da
    return a


def sigmoid(z, derivative=False):
    a = 1/(1+np.exp(-z))
    if derivative:
        da = a * (1 - a)
        return a, da
    return a


def tanh(z, derivative=False):
    a = np.tanh(z)
    if derivative:
        da = (1 - a) * (1 + a)
        return a, da
    return a


def relu(z, derivative=False):
    a = z * (z >= 0)
    if derivative:
        da = np.array(z >= 0, dtype=float)
        return a, da
    return a

Paso 3. Completa la siguiente clase para programar la red densa

In [ ]:
class MLP:

  def __init__(self, layers_dims, 
               hidden_activation=relu,
               output_activation=sigmoid,
               learning_rate=0.1):

      # Instance Attributes
    self.L = len(layers_dims) - 1
    self.w = [None] * (self.L + 1)
    self.b = [None] * (self.L + 1)
    self.f = [None] * (self.L + 1)
    self.layers = layers_dims
    self.eta = learning_rate

    # Initialize weights
    for l in range(1, self.L + 1):
      self.w[l] = -1 + 2 * np.random.rand(layers_dims[l], layers_dims[l-1])
      self.b[l] = -1 + 2 * np.random.rand(layers_dims[l], 1)
            
      if l == self.L:
          self.f[l] = output_activation
      else:
          self.f[l] = hidden_activation
                

  def predict(self, X):
    a = np.asanyarray(X)
    for l in range(1, self.L + 1):
        z = np.dot(self.w[l], a) + self.b[l]
        a = self.f[l](z)
    return a

  def fit(self, X, Y, epochs=500):

    # Number of samples
    p = X.shape[1]

    #Gradient Descent
    for _ in range(epochs):

      # Initialize activations and their derivatives
      A = [None] * (self.L + 1)
      dA = [None] * (self.L + 1)
      lg = [None] * (self.L + 1)
                
      # Propagation
      A[0] = X
      for l in range(1, self.L + 1):
        Z = np.dot(self.w[l], A[l-1]) + self.b[l]
        A[l], dA[l] = self.f[l](Z, derivative=True)

      # Backpropagation
      for l in range(self.L, 0, -1):
        # Calculate local gradient (lg)
        if l == self.L:
          lg[l] = (Y - A[l]) * dA[l]
        else:
          lg[l] = np.dot(self.w[l + 1].T, lg[l + 1]) * dA[l]
                        
      # Update parameters
      for l in range(1, self.L + 1):
        self.w[l] += (self.eta/p) * np.dot(lg[l], A[l - 1].T)
        self.b[l] += (self.eta/p) * np.sum(lg[l])

Paso 4. Ejecuta la siguiente linea que te ayudará a dibujar las predicciones de la red

In [ ]:
def MLP_binary_classification_2d(X,Y,net):
    plt.figure()
    for i in range(X.shape[1]):
        if Y[0,i]==0:
            plt.plot(X[0,i], X[1,i], '.r')
        else:
            plt.plot(X[0,i], X[1,i], '.b')
    xmin, ymin=np.min(X[0,:])-0.5, np.min(X[1,:])-0.5
    xmax, ymax=np.max(X[0,:])+0.5, np.max(X[1,:])+0.5
    xx, yy = np.meshgrid(np.linspace(xmin,xmax,100), 
                         np.linspace(ymin,ymax,100))
    data = [xx.ravel(), yy.ravel()]
    zz = net.predict(data)
    zz = zz.reshape(xx.shape)
    plt.contourf(xx,yy,zz, alpha=0.8, 
                 cmap=plt.cm.RdBu)
    plt.xlim([xmin,xmax])
    plt.ylim([ymin,ymax])
    plt.grid()
    plt.show()

Paso 5. Crea los datos para de la compuerta XOR

In [ ]:
X = np.array([[0, 0, 1, 1],
              [0, 1, 0, 1]])
Y = np.array([[1, 0, 0, 1]]) 

Paso 6. Repite el código anterior pero agrega el entrenamiento

In [ ]:
net.fit(X, Y)
print(net.predict(X))
MLP_binary_classification_2d(X,Y,net)